In [1]:
import numpy as np
from scipy.io import loadmat, wavfile
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib as mpl
from Signal_Analysis.features.signal import get_F_0, get_HNR
from importlib import reload
import wave
import json
from vosk import Model, KaldiRecognizer,SetLogLevel


# tools work in progress
import tools
reload(tools)
from tools import *

plt.style.use("dark_background")
mpl.rcParams["lines.linewidth"] = 0.5
plt.close("all")



In [2]:
#model_path = "models/vosk-model-en-us-0.22"
#model_path = "models/vosk-model-small-en-us-0.15"
model_path = "models/vosk-model-small-sv-rhasspy-0.15/"
if not "model" in locals():
    model = Model(model_path)


In [3]:

class Word:
    ''' A class representing a word from the JSON format for vosk speech recognition API '''

    def __init__(self, dict):
        '''
        Parameters:
          dict (dict) dictionary from JSON, containing:
            conf (float): degree of confidence, from 0 to 1
            end (float): end time of the pronouncing the word, in seconds
            start (float): start time of the pronouncing the word, in seconds
            word (str): recognized word
        '''

        self.conf = dict["conf"]
        self.end = dict["end"]
        self.start = dict["start"]
        self.word = dict["word"]

    def to_string(self):
        ''' Returns a string describing this instance '''
        return "{:20} from {:.2f} sec to {:.2f} sec, confidence is {:.2f}%".format(
            self.word, self.start, self.end, self.conf*100)

In [9]:
#audio_filename = "wav_examples/kaviar_sv.wav"
#audio_filename = "wav_examples/digitala_resurser_sv.wav"
audio_filename = "wav_examples/sammarbete_sv.wav"
wf = wave.open(audio_filename, "rb")

print(wf.getnframes())
print(wf.getframerate())

137088
32000


In [11]:
wf = wave.open(audio_filename, "rb")

rec = KaldiRecognizer(model, wf.getframerate())
rec.SetWords(True)

# get the list of JSON dictionaries
results = []
# recognize speech using vosk model
while True:
    data = wf.readframes(wf.getframerate())
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        part_result = json.loads(rec.Result())
        results.append(part_result)
part_result = json.loads(rec.FinalResult())
results.append(part_result)


# convert list of JSON dictionaries to list of 'Word' objects
list_of_words = []
for sentence in results:
    if len(sentence) == 1:
        # sometimes there are bugs in recognition 
        # and it returns an empty dictionary
        # {'text': ''}
        continue
    for obj in sentence['result']:
        w = Word(obj)  # create custom Word object
        list_of_words.append(w)  # and add it to list

wf.close()  # close audiofile

# output to the screen
for word in list_of_words:
    print(word.to_string())


detta                from 0.75 sec to 1.26 sec, confidence is 100.00%
underlättar          from 1.29 sec to 2.16 sec, confidence is 100.00%
samarbetet           from 2.16 sec to 3.42 sec, confidence is 100.00%


## Test timestamps

In [6]:
list_of_words[0]